In [2]:
import torch #torch 함수들 불러오는 역할
import torch.nn as nn
import torch.onnx #불러온 torch 함수를  onnx로 변환하는 역할
import onnx #onnx 확인
import onnxruntime #onnx 계산
import numpy as np # onnx 계산 할 때는 numpy를 사용

In [3]:
# class 선언

class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.encode_rnn = nn.LSTM(13, 13, batch_first = True)
        self.decode_f = nn.Linear(13, 7)
        
    def encode_forward(self, x) :
        y, hc = self.encode_rnn(x)
        return y, hc        
        
    def decode_forward(self, hc) :
        h = hc[0]
        h = h.reshape(-1, 13)
        y = self.decode_f(h)
        return y
        
    def forward(self, x) :
        y, hc = self.encode_forward(x)
        y = self.decode_forward(hc)
        return y

In [5]:
# torch 불러오기

F = torch.load('sound_ai.pt', weights_only=False).to('cpu') #토치 함수 불러오고, cpu로 변환
F.eval() #dropout 끄기

print(F)

NN(
  (encode_rnn): LSTM(13, 13, batch_first=True)
  (decode_f): Linear(in_features=13, out_features=7, bias=True)
)


In [6]:
# torch를 onnx로 변환
x = torch.randn(1,1234,13) #더미 입력값
dynamic_axes = {'input' : {0 : 'b', 1 : 'f'}, 'output' : {0 : 'b'}} #입력값과 출력값의 가변 차원 (길이가 바뀌어도 계산되는 차원)

torch.onnx.export(
    F,
    x,
    'sound_ai.onnx',
    export_params = True, #w, b 저장
    opset_version = 20,
    do_constant_folding = True,
    input_names = ['input'],
    output_names = ['output'],
    dynamic_axes = dynamic_axes
)

C:\Users\Administrator\anaconda3\Lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [7]:
onnx_F = onnx.load('sound_ai.onnx')
onnx.checker.check_model(onnx_F)

In [15]:
x = torch.randn(2,5,13)
numpy_x = x.detach().numpy()

onnx_F = onnxruntime.InferenceSession('sound_ai.onnx', providers=['CPUExecutionProvider'])

y = F(x)
inputs = {onnx_F.get_inputs()[0].name : numpy_x}
outputs = onnx_F.run(None, inputs)

print(y.detach().numpy())
print()
print(outputs[0])


[[ 0.18746682 -0.6808965   0.4690648   0.48259994 -0.2092566   0.2997513
  -0.6299207 ]
 [ 0.02194116 -0.59072715  0.6736289  -0.05012216  0.07579528 -0.31417677
  -0.44111225]]

[[ 0.18746684 -0.6808965   0.4690649   0.48259994 -0.20925663  0.29975134
  -0.6299208 ]
 [ 0.02194114 -0.5907271   0.6736288  -0.05012216  0.07579522 -0.3141768
  -0.4411122 ]]
